<a href="https://colab.research.google.com/github/LamuGit/FinText-Data/blob/main/Random_Forest_Meta_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
!pip install xlsxwriter
from xlsxwriter import Workbook
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)
import yfinance as yf

# Load the files
df1 = pd.read_excel('/Extracted_data_sentiment_Lamu_FinBert_ProsusAI.xlsx')
df2 = pd.read_excel('/Extracted_data_sentiment_Lamu_Finbert_yiyanghkust.xlsx')
df3 = pd.read_excel('/Extracted_data_sentiment_Lamu_distilroberta_mrm8488.xlsx')

# Define a function to calculate the sentiment score for each sentence
def calculate_sentiment_score(row):
    sentiment_score = row['Positive Probability'] - row['Negative Probability'] + (row['Neutral Probability'] * 0.5)
    return sentiment_score

# Apply the function to each row of the dataframes
df1['Sentiment Score'] = df1.apply(calculate_sentiment_score, axis=1)
df2['Sentiment Score'] = df2.apply(calculate_sentiment_score, axis=1)
df3['Sentiment Score'] = df3.apply(calculate_sentiment_score, axis=1)

# Rename the column 'Sentiment Score' in df3 to 'Sentiment Score_llm3'
df3 = df3.rename(columns={'Sentiment Score': 'Sentiment Score_llm3'})

# Rename the column 'Sentiment Score' in df1 and df2
df1 = df1.rename(columns={'Sentiment Score': 'Sentiment Score_llm1'})
df2 = df2.rename(columns={'Sentiment Score': 'Sentiment Score_llm2'})

# Merge the dataframes on the Sentence column
merged_df = pd.merge(df1, df2, on='Sentence', how='inner')
merged_df = pd.merge(merged_df, df3, on='Sentence', how='inner')

# Define the feature columns (sentiment scores from each model)
X = merged_df[['Sentiment Score_llm1', 'Sentiment Score_llm2', 'Sentiment Score_llm3']]

# Define the target column (average sentiment score)
y = merged_df[['Sentiment Score_llm1', 'Sentiment Score_llm2', 'Sentiment Score_llm3']].mean(axis=1)

# Scale the data using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define hyperparameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
 'max_depth': [None, 5, 10],
 'min_samples_split': [2, 5, 10]
}

# Perform grid search to find optimal hyperparameters
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Train a Random Forest model on the training data using optimal hyperparameters
rf = RandomForestRegressor(**grid_search.best_params_)
rf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf.predict(X_test)

# Evaluate the model using multiple metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}')

# Save the results to a new spreadsheet
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})
results_df.to_excel('results.xlsx', index=False, engine='xlsxwriter')

# Save the model performance metrics to a new spreadsheet
metrics_df = pd.DataFrame({
    'Metric': ['MSE', 'MAE', 'R2'],
    'Value': [mse, mae, r2]
})
metrics_df.to_excel('metrics.xlsx', index=False, engine='xlsxwriter')

# Use the trained model to make predictions on the entire dataset
merged_df['Meta Model Score'] = rf.predict(X_scaled)

# Save the resulting dataframe to a new spreadsheet
merged_df.to_excel('meta_model_scores_update.xlsx', index=False, engine='xlsxwriter')


###################################################
#get stock prices
##################################################
# Define the stock symbols

# Load the meta model scores
meta_model_scores = pd.read_excel('meta_model_scores_update.xlsx')

# Get the stock codes
stock_codes = meta_model_scores['Stock'].unique()

# Get the top 10 stock codes
top_10_stock_codes = meta_model_scores['Stock'].value_counts().head(10).index

# Create an empty list to store the stock price data
stock_price_data_list = []

# Loop through each stock code and get the stock price data for 2016
for stock_code in top_10_stock_codes:
    ticker = yf.Ticker(stock_code)
    hist = ticker.history(start='2016-01-01', end='2024-07-31')
    hist['Stock'] = stock_code
    hist.reset_index(inplace=True)
    hist.rename(columns={'index': 'Date'}, inplace=True)
    stock_price_data_list.append(hist)

# Concatenate the stock price data into a single DataFrame
stock_price_data = pd.concat(stock_price_data_list)

# Convert 'Date' column to datetime objects with utc=True to handle timezone-aware datetimes
stock_price_data['Date'] = pd.to_datetime(stock_price_data['Date'], utc=True)

# Now convert to timezone-naive if necessary
stock_price_data['Date'] = stock_price_data['Date'].dt.tz_localize(None)

# Save the stock price data to a new Excel file
stock_price_data.to_excel('stock_price_data_2016to2024.xlsx', index=False, engine='xlsxwriter')


In [20]:
# Concatenate the stock price data into a single DataFrame
stock_price_data = pd.concat(stock_price_data_list)

# Convert 'Date' column to datetime objects with utc=True to handle timezone-aware datetimes
stock_price_data['Date'] = pd.to_datetime(stock_price_data['Date'], utc=True)

# Now convert to timezone-naive if necessary
stock_price_data['Date'] = stock_price_data['Date'].dt.tz_localize(None)

# Save the stock price data to a new Excel file
stock_price_data.to_excel('stock_price_data_2016.xlsx', index=False, engine='xlsxwriter')

In [19]:
# Concatenate the stock price data into a single DataFrame
stock_price_data = pd.concat(stock_price_data_list)

# Convert 'Date' column to datetime objects
stock_price_data['Date'] = pd.to_datetime(stock_price_data['Date']) # Convert 'Date' to datetime

# Convert 'Date' column to timezone-naive datetime objects (if necessary)
stock_price_data['Date'] = stock_price_data['Date'].dt.tz_localize(None)

# Save the stock price data to a new Excel file
stock_price_data.to_excel('stock_price_data_2016.xlsx', index=False, engine='xlsxwriter')

ValueError: Tz-aware datetime.datetime cannot be converted to datetime64 unless utc=True, at position 252